# Notebook 1
Raw data export into np.memaps.

### Load important modules

In [1]:
# Standard modules
import numpy as np
import os
import logging

import pickle

# Move to root directory for easier module handling
os.chdir("../..")
print(os.listdir("."))
from notebooks.data_processing.modules.maldi_conversion import process_raw_data, extract_raw_data
from notebooks.data_processing.modules.lookup_tables import process_lookup_tables
from modules.tools.misc import delete_all_files_in_folder

# multithreading/multiprocessing
from multiprocessing import Pool
from threadpoolctl import threadpool_limits

# set thread limit
threadpool_limits(16)


['data', 'pages', 'app.py', 'assets', 'config.py', 'index.py', 'main.py', 'TODO.py', 'notebooks', 'modules', '__pycache__']


### Create a list of raw data filenames

In [2]:
# Load filenames
l_t_names = sorted(
    [
        [
            int(name.split("MouseBrainCMC_S")[1].split("_")[0].split("A")[0].split("(")[0]),
            "/data/lipidatlas/data/data_raw/" + name + "/" + name,
        ]
        for name in os.listdir("/data/lipidatlas/data/data_raw/")
        if "MouseBrain" in name
    ]
)

# Correct for duplicates
for t_names_1, t_names_2 in zip(l_t_names[:-1], l_t_names[1:]):
    if t_names_2[0] == t_names_1[0]:
        t_names_2.append("bis")
        print("WARNING: duplicate for slice " + str(t_names_1[0]))

# Remove slices that have already been processed
path = "notebooks/data_processing/data/temp/"
os.makedirs(path, exist_ok=True)
remove_already_loaded = False
if remove_already_loaded:
    existing_names = [int(name.split("_")[1][:-7]) for name in os.listdir(path) if "raw" in name]
    l_t_names = [x for x in l_t_names if x[0] not in existing_names]

# Print the final list of names
for t_names in l_t_names:
    print(t_names[0], t_names[1].split("/")[-1])


1 20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um
2 20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um
3 20210213_MouseBrainCMC_S3AC4_2Dpixelmode_371x195_Att25_25um
4 20210214_MouseBrainCMC_S4AD3_2Dpixelmode_354x228_Att25_25um
5 20210218_MouseBrainCMC_S5AE3_2Dpixelmode_396x272_Att25_25um
6 20210219_MouseBrainCMC_S6AE3_2Dpixelmode_423x282_Att25_25um
7 20210220_MouseBrainCMC_S7AF5_2Dpixelmode_427x263_Att25_25um
8 20210531_MouseBrainCMC_S8_duplicate_2Dpixelmode_430x285_Att30_25um
9 20210224_MouseBrainCMC_S9AH4_2Dpixelmode_467x278_Att25_25um
10 20210210_MouseBrainCMC_S10(brain2_20)_394x282_Att30_25um
11 20210301_MouseBrainCMC_S11AK5_2Dpixelmode_448x277_Att25_25um
12 20210303_MouseBrainCMC_S12AL1_2Dpixelmode_393x266_Att25_25um
13 20210304_MouseBrainCMC_S13AM1_2Dpixelmode_413x310_Att25_25um
14 20210305_MouseBrainCMC_S14AN1_2Dpixelmode_409x285_Att25_25um
15 20210313_MouseBrainCMC_S15AO2_2Dpixelmode_451x292_Att25_25um
16 20210530_MouseBrainCMC_S16_duplicate_2Dpixelmode_454


### Extract raw data into numpy arrays with multiprocessing

In [ ]:
# multiprocessing = False
# if multiprocessing:
#     with Pool(processes=14) as pool:
#         [x for x in pool.imap_unordered(extract_raw_data, l_t_names)]
# else:
#     # Normal (single-processed) map
#     [x for x in map(extract_raw_data, l_t_names)]

### Process raw data into numpy arrays with multiprocessing

In [3]:
multiprocessing = True
if multiprocessing:
    with Pool(processes=16) as pool:
        [x for x in pool.imap_unordered(process_raw_data, l_t_names)]
else:
    # Normal (single-processed) map
    [x for x in map(process_raw_data, l_t_names)]


Compute and normalize pixels values according to TIC
Filtering out noise and matrix peaks
Sorting by m/z value for averaging before standardization
Getting spectrums array averaged accross pixels
Prepare data for standardization
Standardize data
Compute and normalize pixels values according to TIC
259 260 0.00020909094018861651 0.00020909107788173194 0.7323678265553686
261 262 0.00023867991694714874 0.00023867981542903581 0.7064667092510108
260 261 0.000220037778490223 0.00022003785506540088 0.6592818897258698
246 247 0.0002658390731085092 0.00026583909108768473 0.7541586468642276
242 243 0.00024476039106957614 0.00024476022940102005 0.7619305478315267
259 260 0.0002585422189440578 0.0002585427014217025 0.7266627927932897
236 237 0.0002806723932735622 0.00028067202322002555 0.6870055776594983
257 258 0.00023221634910441935 0.00023221643880791247 0.8508497356713519
192 193 0.00015526567585766315 0.00015526564457235746 0.8090616897674328
6 7 0.0008591866935603321 0.0009368371136991118 0.

/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:556: RuntimeWarning: divide by zero encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)
/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:556: RuntimeWarning: invalid value encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)


Sorting by m/z value for averaging
Filtering out noise and matrix peaks
Sorting by m/z value for averaging before standardization
Getting spectrums array averaged accross pixels
Prepare data for standardization
Compute and normalize pixels values according to TIC
Sorting by m/z value for averaging before standardization
Getting spectrums array averaged accross pixels
Prepare data for standardization
Compute and normalize pixels values according to TIC
Filtering out noise and matrix peaks
Getting spectrums array averaged accross pixels
Sorting by m/z value for averaging before standardization
Build the low-resolution averaged array from the high resolution averaged array
Getting spectrums array averaged accross pixels
Double sorting according to pixel and mz high-res array
Filtering out noise and matrix peaks
Prepare data for standardization
Compute and normalize pixels values according to TIC
Compute and normalize pixels values according to TIC
Filtering out noise and matrix peaks
Gett

/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:556: RuntimeWarning: divide by zero encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)
/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:556: RuntimeWarning: invalid value encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)


152 153 0.0005593067035079002 0.0005593066697345533 0.6565821428987713
Sorting by m/z value for averaging
169 170 0.0005222613108344376 0.0005222610546084389 0.7155247865644582
242 243 0.00047499826177954674 0.00047499813498811825 0.7334216009787746
193 194 0.0005526317399926484 0.0005526315999058041 0.6539835462734332
193 194 0.000645816617179662 0.0006458163693645641 0.746555827398627
184 185 0.0004394371062517166 0.00043943742689775975 1.4001555340558915
135 136 0.0005773706361651421 0.0005773707863894767 1.2124723968339228
181 182 0.0005951510975137353 0.0005951510435942118 0.7987352681537834
169 170 0.0005345893441699445 0.0005345900177230406 0.7677911734042524
208 209 0.0004172151966486126 0.00041721490867238826 0.775364909467155
178 179 0.0006008964264765382 0.0006008965523233876 0.6474876498088503
179 180 0.0005092447972856462 0.0005092443534425429 1.245695989568349
151 152 0.0007271288777701557 0.0007271290479905223 0.7344295887785274
166 167 0.0006377596291713417 0.0006377594

...

Sorting by m/z value for averaging
222 223 0.00017803760420065373 0.00017803787504757273 0.6570495145304921
201 202 0.00015653074660804123 0.00015653066087734785 0.5812638268333137
299 300 0.00021153924171812832 0.00021153929939623332 1.454887043741065
244 245 0.0003211548610124737 0.0003211549659883219 1.4834419699791488
254 255 0.0001265893515665084 0.00012658946546251768 0.5314215037462263
207 208 0.00012945651542395353 0.0001294565849216277 0.5334473747395513
283 284 0.00015992512635421008 0.00015992508208867798 1.5972216851163665
266 267 0.0002749153063632548 0.00027491539327193534 1.3790714775262973
202 203 0.0001698770938673988 0.00016987710360018103 0.7100950713409826
269 270 0.00023707006766926497 0.0002370701535647207 1.5367374218840684
209 210 0.00018841464770957828 0.00018841467180298648 0.6222342311671614
308 309 0.0001539144868729636 0.00015391447453576003 1.6165215535747022
191 192 0.00013588838919531554 0.00013588834733316692 0.541018757511502
286 287 0.0002644154883455

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Standardize data
Getting spectrums array averaged accross pixels
Filtering out noise and matrix peaks
Prepare data for standardization
554 555 0.00026654399698600173 0.0002665438909937314 0.12185995299169657
360 361 0.000260584318311885 0.0002605844336916013 0.12464694404234092
224 225 0.000259850436123088 0.0002598504265434911 0.8868116580441744
215 216 0.00025523125077597797 0.00025523161741433604 1.2937031104559242
219 220 0.00023299372696783394 0.00023299314348318537 0.0435488834477001
253 254 0.0001775517303030938 0.00017755180151896401 0.08432172835891893
247 248 0.00025848549557849765 0.0002584858096452776 1.0162464783515026
283 284 0.00018366494623478502 0.00018366499732818238 0.04324427345739868
291 292 0.00024192375713028014 0.00024192409235982473 0.03298931017462147
279 280 0.0002600692387204617 0.00026006928060431196 1.1228529469463158
128 129 0.0002742937649600208 0.0002742939866576702 2.3646399202432935
204 205 0.0002893288619816303 0.0002893287489502647 0.915442543322458

/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:556: RuntimeWarning: divide by zero encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)
/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:556: RuntimeWarning: invalid value encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)


Sorting by m/z value for averaging
Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Double sorting according to pixel and mz high-res array
Getting corresponding spectra arrays
Saving : /data/lipidatlas/data/data_raw/20210423_MouseBrainCMC_S28_3_2Dpixelmode_390x244_Att30_25um/20210423_MouseBrainCMC_S28_3_2Dpixelmode_390x244_Att30_25um
Getting spectrums array averaged accross pixels
Sorting by m/z value for averaging before standardization
Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Double sorting according to pixel and mz high-res array
Build the low-resolution averaged array from the high resolution averaged array
Double sorting according to pixel and mz high-res array
Getting corresponding spectra arrays
Saving : /data/lipidatlas/data/data_raw/20210504_MouseBrainCMC_S32_3_2Dpixelmode_298x230_Att30_25um/20210504_MouseBrainCM

### Build lookup tables

In [4]:
multiprocessing = True
if multiprocessing:
    # Multiprocessing
    with Pool(processes=16) as pool:
        [x for x in pool.map(process_lookup_tables, l_t_names)]
else:
    # Normal (single-processed) map
    [x for x in map(process_lookup_tables, l_t_names)]



Size (in mb) of lookup_table_spectra_high_res:  551.95
Shape of lookup_table_spectra_high_res:  (2000, 72345)
Size (in mb) of lookup_table_spectra_high_res:  604.1
Shape of lookup_table_spectra_high_res:  (2000, 79180)
Size (in mb) of lookup_table_spectra_high_res:  615.78
Shape of lookup_table_spectra_high_res:  (2000, 80712)
Size (in mb) of lookup_table_spectra_high_res:  821.78
Shape of lookup_table_spectra_high_res:  (2000, 107712)
Size (in mb) of lookup_table_spectra_high_res:  567.49
Shape of lookup_table_spectra_high_res:  (2000, 74382)
Size (in mb) of lookup_table_spectra_high_res:  910.08
Shape of lookup_table_spectra_high_res:  (2000, 119286)
Size (in mb) of lookup_table_spectra_high_res:  797.56
Shape of lookup_table_spectra_high_res:  (2000, 104538)
Size (in mb) of lookup_table_spectra_high_res:  889.32
Shape of lookup_table_spectra_high_res:  (2000, 116565)
Size (in mb) of lookup_table_spectra_high_res:  847.69
Shape of lookup_table_spectra_high_res:  (2000, 111108)
Size (

### Record everything and clean 

Record everything in memap files and a pickled dictonnary

In [5]:
output_folder = "data/whole_dataset/"
input_folder = "notebooks/data_processing/data/temp/"
os.makedirs(output_folder, exist_ok=True)

dic_slices = {}
# Loop over slice files
for slice_name in os.listdir(input_folder):
    print(slice_name)
    if 'raw' in slice_name or 'checkpoints' in slice_name:
        continue
    # Extract slice index
    slice_index = int(slice_name.split("_")[1][:-4])

    # Load slice arrays
    npzfile = np.load(input_folder + slice_name)
    array_pixel_indexes_high_res = npzfile["array_pixel_indexes_high_res"]
    array_spectra_high_res = npzfile["array_spectra_high_res"]
    array_averaged_mz_intensity_low_res = npzfile["array_averaged_mz_intensity_low_res"]
    array_averaged_mz_intensity_high_res = npzfile["array_averaged_mz_intensity_high_res"]
    array_averaged_mz_intensity_high_res_before_standardization = npzfile["array_averaged_mz_intensity_high_res_before_standardization"]
    image_shape = npzfile["image_shape"]
    divider_lookup = npzfile["divider_lookup"]
    lookup_table_spectra_high_res = npzfile["lookup_table_spectra_high_res"]
    cumulated_image_lookup_table_high_res = npzfile["cumulated_image_lookup_table_high_res"]
    lookup_table_averaged_spectrum_high_res = npzfile["lookup_table_averaged_spectrum_high_res"]
    array_peaks_corrected = npzfile["array_peaks_corrected"]
    array_corrective_factors = npzfile["array_corrective_factors"]

    # Print array size
    # print size used by each array in mb
    print(round(array_pixel_indexes_high_res.nbytes / 1024 / 1024, 2))
    print(round(array_spectra_high_res.nbytes / 1024 / 1024, 2))
    print(round(array_averaged_mz_intensity_low_res.nbytes / 1024 / 1024, 2))
    print(round(array_averaged_mz_intensity_high_res.nbytes / 1024 / 1024, 2))
    print(round(lookup_table_spectra_high_res.nbytes / 1024 / 1024, 2))
    print(round(cumulated_image_lookup_table_high_res.nbytes / 1024 / 1024, 2))
    print(round(lookup_table_averaged_spectrum_high_res.nbytes / 1024 / 1024, 2))

    # Register the lightweights files in a pickled dictionnary
    dic_slices[slice_index] = {
        "image_shape": image_shape,
        "divider_lookup": divider_lookup,
        "array_avg_spectrum_downsampled": array_averaged_mz_intensity_low_res,
        "array_lookup_pixels": array_pixel_indexes_high_res,
        "array_lookup_mz_avg": lookup_table_averaged_spectrum_high_res,
        "array_peaks_transformed_lipids": array_peaks_corrected,
        "array_corrective_factors": array_corrective_factors,
    }

    # Build a memap for each of the heavier files to save RAM, save the corresponding shape in the
    # pickled dictionnary
    fp = np.memmap(
        output_folder + "array_spectra_" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=array_spectra_high_res.shape,
    )
    fp[:] = array_spectra_high_res[:]
    fp.flush()
    dic_slices[slice_index]["array_spectra_shape"] = array_spectra_high_res.shape

    fp = np.memmap(
        output_folder + "array_avg_spectrum_" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=array_averaged_mz_intensity_high_res.shape,
    )
    fp[:] = array_averaged_mz_intensity_high_res[:]
    fp.flush()
    dic_slices[slice_index]["array_avg_spectrum_shape"] = array_averaged_mz_intensity_high_res.shape

    fp = np.memmap(
        output_folder + "array_avg_spectrum_before_standardization" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=array_averaged_mz_intensity_high_res_before_standardization.shape,
    )
    fp[:] = array_averaged_mz_intensity_high_res_before_standardization[:]
    fp.flush()
    dic_slices[slice_index]["array_avg_spectrum_before_standardization_shape"] = array_averaged_mz_intensity_high_res_before_standardization.shape


    fp = np.memmap(
        output_folder + "array_lookup_mz_" + str(slice_index) + ".mmap",
        dtype="int32",
        mode="w+",
        shape=lookup_table_spectra_high_res.shape,
    )
    fp[:] = lookup_table_spectra_high_res[:]
    fp.flush()
    dic_slices[slice_index]["array_lookup_mz_shape"] = lookup_table_spectra_high_res.shape

    fp = np.memmap(
        output_folder + "array_cumulated_lookup_mz_image_" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=cumulated_image_lookup_table_high_res.shape,
    )
    fp[:] = cumulated_image_lookup_table_high_res[:]
    fp.flush()
    dic_slices[slice_index][
        "array_cumulated_lookup_mz_image_shape"
    ] = cumulated_image_lookup_table_high_res.shape

# Pickle the dict of lightweight data
with open(output_folder + "light_arrays.pickle", "wb") as handle:
    pickle.dump(dic_slices, handle)


slice_14raw.npz
slice_8raw.npz
slice_26raw.npz
slice_13raw.npz
slice_15raw.npz
slice_16raw.npz
slice_21raw.npz
slice_22raw.npz
slice_23raw.npz
slice_12raw.npz
slice_30raw.npz
slice_6raw.npz
slice_9raw.npz
slice_31raw.npz
slice_1.npz
0.57
148.1
0.05
0.98
567.49
567.49
0.01
slice_2.npz
0.6
158.53
0.06
1.1
604.1
604.1
0.01
slice_3.npz
0.55
71.81
0.05
0.94
551.95
551.95
0.01
slice_4.npz
0.62
61.66
0.05
1.32
615.78
615.78
0.01
slice_5.npz
0.82
136.45
0.05
1.13
821.78
821.78
0.01
slice_10.npz
0.85
273.05
0.06
1.35
847.69
847.69
0.01
slice_7.npz
0.86
218.57
0.06
1.24
856.79
856.79
0.01
slice_12.npz
0.8
226.24
0.06
1.21
797.56
797.56
0.01
slice_6.npz
0.91
210.06
0.06
1.32
910.08
910.08
0.01
slice_11.npz
0.95
251.87
0.06
1.25
946.78
946.78
0.01
slice_8.npz
0.93
162.56
0.06
1.2
934.98
934.98
0.01
slice_9.npz
0.99
270.59
0.06
1.36
990.49
990.49
0.01
slice_14.npz
0.89
223.25
0.06
1.24
889.32
889.32
0.01
slice_13.npz
0.98
232.6
0.08
1.8
976.79
976.79
0.01
slice_19.npz
0.7
144.88
0.05
1.12
700.93
70

Clean temporary folder

In [ ]:
clean = False
if clean:
    delete_all_files_in_folder(input_folder)
